In [31]:
!pip install polars

  Obtaining dependency information for polars from https://files.pythonhosted.org/packages/19/9d/78a6d5f4961587f3f0776623122782ce0c902f82ba5917f42883e3fa08e4/polars-0.20.8-cp38-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/25.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/25.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/25.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/25.7 MB 495.5 kB/s eta 0:00:52
   ---------------------------------------- 0.1/25.7 MB 1.2 MB/s eta 0:00:21
    --------------------------------------- 0.5/25.7 MB 2.6 MB/s eta 0:00:10
   - -------------------------------------- 0.8/25.7 MB 3.9 MB/s eta 0:00:07
   - -------------------------------------- 1.0/25.7 MB 3.9 MB/s eta 0:00:07
   -- ------------------------------------- 1.3/25.7 MB 4.2 MB/s eta 0:00:06
   -- ------------------------------------- 1.8/25.7 MB 5.0 MB/s eta 0:00:05
   --- ------------------------------------ 

In [3]:
import open3d as o3d
print(o3d.__version__)
import numpy as np
import pandas as pd
import polars as pl

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
0.18.0


In [4]:
print("kota_circuit")
ply_point_cloud = o3d.data.PLYPointCloud()
pcd = o3d.io.read_point_cloud("Dataset/kota_circuit2.ply")
print(pcd)
print(np.asarray(pcd.points))

kota_circuit
PointCloud with 14640197 points.
[[  82.07930756 -147.18885803  -78.15903473]
 [  97.60070801  149.98822021  -53.49351883]
 [  70.49710846 -229.24028015  -78.54177856]
 ...
 [  26.64775658  -36.49408722  -89.59648132]
 [ -88.472229     49.91682816  -75.39753723]
 [ -23.17474937  335.25888062 -102.58750916]]


In [20]:
print("render file")
o3d.visualization.draw_geometries([pcd],
                                  zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024])

render file


In [21]:
# for cropping area by manual -> cropped_kota_circuit
# o3d.visualization.draw_geometries_with_editing([pcd])
crop_pcd = o3d.io.read_point_cloud("Dataset/cropped_kota_circuit2.ply")
o3d.visualization.draw_geometries([crop_pcd])

In [6]:
print(crop_pcd)
print(np.asarray(crop_pcd.points))

PointCloud with 69665 points.
[[ 15.44811535 283.57678223 -94.7826004 ]
 [  4.93768883 299.45748901 -94.67481995]
 [ 22.11404228 298.99884033 -85.44134521]
 ...
 [ 16.59041595 281.490448   -94.74358368]
 [ 17.36798668 296.38348389 -94.73506165]
 [ 23.36079216 294.47653198 -86.0475769 ]]


In [22]:
print("Normal Vector")
crop_pcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
o3d.visualization.draw_geometries([crop_pcd],
                                  point_show_normal=True)

Normal Vector


In [8]:
# Setting
point_cloud = np.asarray(crop_pcd.points) 
color_point = np.asarray(crop_pcd.colors)
normal_vector = np.asarray(crop_pcd.normals)

print("point cloud: \n", point_cloud)
print()
print("normal vector: \n", normal_vector)
print()
print("color point: \n", color_point)

point cloud: 
 [[ 15.44811535 283.57678223 -94.7826004 ]
 [  4.93768883 299.45748901 -94.67481995]
 [ 22.11404228 298.99884033 -85.44134521]
 ...
 [ 16.59041595 281.490448   -94.74358368]
 [ 17.36798668 296.38348389 -94.73506165]
 [ 23.36079216 294.47653198 -86.0475769 ]]

normal vector: 
 [[-0.32016957  0.06167363 -0.94535063]
 [ 0.          0.          1.        ]
 [ 0.          0.          1.        ]
 ...
 [ 0.87666944  0.19223679  0.44101668]
 [ 0.          0.          1.        ]
 [ 0.03116634 -0.91377031 -0.40503393]]

color point: 
 [[0.90980392 0.90980392 0.92156863]
 [0.74509804 0.74509804 0.59215686]
 [0.30196078 0.35294118 0.21960784]
 ...
 [0.75294118 0.76862745 0.58039216]
 [0.66666667 0.65490196 0.49803922]
 [0.50980392 0.57647059 0.43137255]]


In [9]:
# Save to .ply in ASCII
output_file_path = "Dataset/cropped_ASCII.ply"

o3d.io.write_point_cloud(output_file_path, crop_pcd, write_ascii=True)

True

In [10]:
# set to Dataframe
file_path = 'Dataset/cropped_ASCII.ply'

data = []
check = False

with open(file_path, 'r') as file:
    for line in file:
        if check :
            row = line.strip().split()
            data.append(row)
        elif "end_header" in line:
            check = True
            
df = pl.DataFrame(data, schema=['x','y','z','nx','ny','nz','r','g','b'])
print(df)

shape: (69_665, 9)
┌─────────┬─────────┬──────────┬───────────┬───┬───────────┬─────┬─────┬─────┐
│ x       ┆ y       ┆ z        ┆ nx        ┆ … ┆ nz        ┆ r   ┆ g   ┆ b   │
│ ---     ┆ ---     ┆ ---      ┆ ---       ┆   ┆ ---       ┆ --- ┆ --- ┆ --- │
│ str     ┆ str     ┆ str      ┆ str       ┆   ┆ str       ┆ str ┆ str ┆ str │
╞═════════╪═════════╪══════════╪═══════════╪═══╪═══════════╪═════╪═════╪═════╡
│ 15.4481 ┆ 283.577 ┆ -94.7826 ┆ -0.32017  ┆ … ┆ -0.945351 ┆ 232 ┆ 232 ┆ 235 │
│ 4.93769 ┆ 299.457 ┆ -94.6748 ┆ 0         ┆ … ┆ 1         ┆ 190 ┆ 190 ┆ 151 │
│ 22.114  ┆ 298.999 ┆ -85.4413 ┆ 0         ┆ … ┆ 1         ┆ 77  ┆ 90  ┆ 56  │
│ 22.3931 ┆ 269.564 ┆ -94.7922 ┆ 0.0808842 ┆ … ┆ 0.792344  ┆ 182 ┆ 190 ┆ 189 │
│ 10.9315 ┆ 287.34  ┆ -94.6872 ┆ 0         ┆ … ┆ 1         ┆ 217 ┆ 221 ┆ 225 │
│ …       ┆ …       ┆ …        ┆ …         ┆ … ┆ …         ┆ …   ┆ …   ┆ …   │
│ 14.9662 ┆ 300.093 ┆ -88.1495 ┆ 0.0450678 ┆ … ┆ -0.872844 ┆ 98  ┆ 121 ┆ 90  │
│ 31.3702 ┆ 301.127 ┆ -87.6172 ┆ 

In [26]:
filter_crop = df.filter(
    # with normal vector
    (df['nz'] >= 0.9) &
    (-0.2<df['ny']) & (df['ny']<0.2) &
    (-0.2<df['nx']) & (df['nx']<0.2) &
    # with color
    ~( (df['g']>=df['b']) & (df['g']>=df['r']) ) 
    # ( (df['r']<=90) & (df['g']<=90) & (df['b']<=90) )
    # (df['g']-df['r']<30) & 
    # (df['g']-df['b']<30)
)

print(filter_crop)

shape: (22_944, 9)
┌─────────┬─────────┬──────────┬─────┬───┬─────┬─────┬─────┬─────┐
│ x       ┆ y       ┆ z        ┆ nx  ┆ … ┆ nz  ┆ r   ┆ g   ┆ b   │
│ ---     ┆ ---     ┆ ---      ┆ --- ┆   ┆ --- ┆ --- ┆ --- ┆ --- │
│ str     ┆ str     ┆ str      ┆ str ┆   ┆ str ┆ str ┆ str ┆ str │
╞═════════╪═════════╪══════════╪═════╪═══╪═════╪═════╪═════╪═════╡
│ 10.9315 ┆ 287.34  ┆ -94.6872 ┆ 0   ┆ … ┆ 1   ┆ 217 ┆ 221 ┆ 225 │
│ 29.1193 ┆ 300.459 ┆ -88.9445 ┆ 0   ┆ … ┆ 1   ┆ 96  ┆ 107 ┆ 90  │
│ 11.8949 ┆ 289.82  ┆ -94.8386 ┆ 0   ┆ … ┆ 1   ┆ 154 ┆ 151 ┆ 154 │
│ 15.2897 ┆ 298.695 ┆ -89.0619 ┆ 0   ┆ … ┆ 1   ┆ 92  ┆ 110 ┆ 85  │
│ 9.16804 ┆ 290.979 ┆ -94.7592 ┆ 0   ┆ … ┆ 1   ┆ 207 ┆ 206 ┆ 203 │
│ …       ┆ …       ┆ …        ┆ …   ┆ … ┆ …   ┆ …   ┆ …   ┆ …   │
│ 19.116  ┆ 296.825 ┆ -86.1207 ┆ 0   ┆ … ┆ 1   ┆ 112 ┆ 135 ┆ 81  │
│ 26.9506 ┆ 283.775 ┆ -94.7324 ┆ 0   ┆ … ┆ 1   ┆ 223 ┆ 226 ┆ 229 │
│ 26.1628 ┆ 300.524 ┆ -88.0016 ┆ 0   ┆ … ┆ 1   ┆ 110 ┆ 110 ┆ 92  │
│ 20.6588 ┆ 285.131 ┆ -94.7867 ┆ 0   ┆ … ┆ 

In [27]:
# Credit by Kit

with open("Dataset/filtered_crop.ply", 'w') as file:
    with open("Dataset/cropped_ASCII.ply", 'r') as ori_file:
        for line in ori_file:
            if "end_header" in line:
                file.write(line)
                ori_file.close()
                break
            else:
                if "element vertex" in line:
                    file.write("element vertex {}\n".format(filter_crop.shape[0]))
                    continue
                file.write(line)
    for row in filter_crop.to_dicts():
        file.write(str(row['x']) + ' ')
        file.write(str(row['y']) + ' ')
        file.write(str(row['z']) + ' ')
        file.write(str(row['nx']) + ' ')
        file.write(str(row['ny']) + ' ')
        file.write(str(row['nz']) + ' ')
        file.write(str(row['r']) + ' ')
        file.write(str(row['g']) + ' ')
        file.write(str(row['b']) + '\n')

In [29]:
path_filter_crop = "Dataset/filtered_crop.ply"

filter_crop_pcd = o3d.io.read_point_cloud(path_filter_crop)
print(filter_crop_pcd)
o3d.visualization.draw_geometries([filter_crop_pcd])

PointCloud with 22944 points.
